## Тестируем ai-intervier

In [43]:
from openai import OpenAI
import json

In [44]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [45]:
SYSTEM_PROMPT_QUESTIONS = """Вы эксперт в составлении последующих вопросов для выявления более глубоких инсайтов."""

SYSTEM_PROMPT_ANALYSIS = """Вы эксперт в анализе коммуникативных навыков из транскриптов интервью. Ваша задача:
1. Проанализировать коммуникативные навыки, продемонстрированные в транскрипте
2. Выявить конкретные цитаты, подтверждающие ваш анализ
3. Предоставить подробный анализ сильных сторон и областей для улучшения"""

In [46]:
def format_questions_prompt(name, number, context):
    return f"""Сгенерируйте {number} технических вопросов для собеседования на позицию: {name}

Контекст: {context}

Требования к вопросам:
- Оценивают технические знания и опыт
- Краткие (до 25 слов)
- Практические примеры
- Открытые вопросы

Верните ТОЛЬКО валидный JSON в точном формате:
{{
  "description": "Техническое собеседование для оценки навыков разработки",
  "questions": [
    {{"question": "Первый вопрос?"}},
    {{"question": "Второй вопрос?"}},
    {{"question": "Третий вопрос?"}},
    {{"question": "Четвертый вопрос?"}},
    {{"question": "Пятый вопрос?"}}
  ]
}}"""

def format_analysis_prompt(transcript):
    return f"""Проанализируйте коммуникативные навыки, продемонстрированные в следующем транскрипте интервью:

Транскрипт: ${transcript}

Пожалуйста, предоставьте ваш анализ в следующем JSON-формате:
{{
  "communicationScore": number, // Оценка от 0 до 10 на основе стандартной системы оценки коммуникации
  "overallFeedback": string,   // 2-3 предложения, обобщающие коммуникативные навыки
  "supportingQuotes": [        // Массив релевантных цитат с анализом
    {{
      "quote": string,         // Точная цитата из транскрипта
      "analysis": string,      // Краткий анализ того, что эта цитата демонстрирует о коммуникативных навыках
      "type": string          // Либо "strength" (сильная сторона), либо "improvement_area" (область для улучшения)
    }}
  ],
  "strengths": [string],       // Список продемонстрированных сильных сторон коммуникации
  "improvementAreas": [string] // Список областей, где коммуникация может быть улучшена
}}
"""

In [60]:
def generate_questions(name, number, context):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_QUESTIONS},
        {"role": "user", "content": format_questions_prompt(name, number, context)}
    ]
    
    response = client.chat.completions.create(
        model="Qwen/Qwen3-8B",
        messages=messages,
        temperature=0.3,  # Снижаем температуру для более стабильного JSON
        max_tokens=800
    )
    
    # Отладочная информация
    raw_response = response.choices[0].message.content
    print("=== ОТЛАДКА: Ответ модели ===")
    print(raw_response)
    print("=== КОНЕЦ ОТЛАДКИ ===\n")
    
    try:
        return json.loads(raw_response)
    except json.JSONDecodeError as e:
        print(f"Ошибка парсинга JSON: {e}")
        # Попробуем извлечь JSON из ответа, если он обернут в текст
        import re
        json_match = re.search(r'\{.*\}', raw_response, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        
        # Фоллбэк: возвращаем заглушку
        return {
            "description": "Техническое собеседование для Senior Python Developer",
            "questions": [
                {"question": "Расскажите о вашем опыте с Django и создании REST API"},
                {"question": "Как вы проектируете архитектуру микросервисов?"},
                {"question": "Опишите процесс оптимизации производительности PostgreSQL"},
                {"question": "Какой опыт у вас с Docker и контейнеризацией?"},
                {"question": "Как вы обеспечиваете качество кода и тестирование?"}
            ]
        }
    
    return json.loads(response.choices[0].message.content)

def analyze_interview(transcript):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_ANALYSIS},
        {"role": "user", "content": format_analysis_prompt(transcript)}
    ]
    
    response = client.chat.completions.create(
        model="Qwen/Qwen3-8B",
        messages=messages,
        temperature=0.5,
        max_tokens=1000
    )
    
    raw_response = response.choices[0].message.content
    
    try:
        # Попытка декодировать JSON
        return json.loads(raw_response)
    except json.JSONDecodeError:
        # Если ответ не является валидным JSON, выводим ошибку и возвращаем None или пустой словарь
        print(f"Ошибка декодирования JSON. Ответ от модели: {raw_response}")
        return None # или верните пустую структуру, например, {}

In [58]:
def format_qa_for_analysis(questions, answers):
    """Форматирование вопросов и ответов для анализа"""
    formatted = ""
    for i, (q, a) in enumerate(zip(questions, answers), 1):
        formatted += f"Интервьюер: {q}\nКандидат: {a}\n\n"
    return formatted

In [49]:
POSITION = "Senior Python Developer"
CONTEXT = "Опыт работы с Django, REST API, PostgreSQL, Docker. Знание архитектурных паттернов и микросервисов."
QUESTIONS_COUNT = 5

if __name__ == "__main__":
    print("=== ГЕНЕРАЦИЯ ВОПРОСОВ ===")
    print(f"Позиция: {POSITION}")
    print(f"Требования: {CONTEXT}")
    print("\n" + "="*50)
    
    # Генерация вопросов
    questions_data = generate_questions(POSITION, QUESTIONS_COUNT, CONTEXT)
    
    print(f"Описание: {questions_data['description']}")
    print("\nВопросы:")
    for i, q in enumerate(questions_data['questions'], 1):
        print(f"{i}. {q['question']}")
    
    print("\n" + "="*50)
    print("Теперь дайте эти вопросы AI-ассистенту для генерации ответов!")

=== ГЕНЕРАЦИЯ ВОПРОСОВ ===
Позиция: Senior Python Developer
Требования: Опыт работы с Django, REST API, PostgreSQL, Docker. Знание архитектурных паттернов и микросервисов.

=== ОТЛАДКА: Ответ модели ===
<think>
Хорошо, мне нужно сгенерировать 5 технических вопросов для собеседования на позицию Senior Python Developer. Контекст указывает на опыт работы с Django, REST API, PostgreSQL, Docker, а также знание архитектурных паттернов и микросервисов. Требования к вопросам: они должны оценивать технические знания и опыт, быть краткими (до 25 слов), практическими и открытыми.

Начну с Django. Важно проверить понимание его архитектуры. Например, можно спросить о том, как реализовать кэширование в Django, чтобы показать знание производительности и архитектурных паттернов.

Далее, REST API. Нужно вопрос, который оценит опыт в проектировании API. Возможно, спросить о методах для безопасного обновления ресурса, что касается HTTP-методов и безопасности.

PostgreSQL требует вопроса о производительно

In [61]:
# === ДАННЫЕ ДЛЯ АНАЛИЗА ===
POSITION = "Senior Python Developer"

# Заполните после получения от AI-ассистента
QUESTIONS = [
    "1. Как реализовать кэширование в Django для улучшения производительности API?",
    "2. Какие HTTP-методы использовать для безопасного обновления ресурса в REST API?", 
    "3. Как оптимизировать PostgreSQL-запрос с JOIN и большим объемом данных?",
    "4. Как настроить Docker-контейнеры для микросервиса с разделением базы данных и приложения?",
    "5. Как интегрировать микросервисы с Django при использовании архитектурного паттерна CQRS?"
]

ANSWERS = [
    "В Django можно использовать встроенную систему кэширования. Настраиваю Redis в settings.py как CACHES, потом использую cache.set() и cache.get() для сохранения результатов запросов. Еще можно использовать декораторы @cache_page для view-функций. Главное не забыть инвалидировать кэш при обновлении данных через cache.clear().",
    
    "Для безопасного обновления ресурсов в REST API нужно использовать PUT для полного обновления объекта и PATCH для частичного. GET только для чтения, POST для создания новых ресурсов. Важно всегда проверять права доступа пользователя перед изменением данных и использовать CSRF токены для защиты от атак.",
    
    "Для оптимизации PostgreSQL запросов с JOIN сначала анализирую план выполнения через EXPLAIN ANALYZE. Добавляю индексы на колонки, которые участвуют в JOIN и WHERE условиях. Использую INNER JOIN вместо LEFT JOIN где возможно, потому что он быстрее. Иногда помогает разбить сложный запрос на несколько простых.",
    
    "Создаю отдельные Dockerfile для приложения и базы данных. В docker-compose.yml настраиваю два сервиса: app и db. Приложение подключается к базе через environment переменные. Важно использовать volumes для персистентности данных БД и networks для изоляции контейнеров. База данных должна запускаться первой, поэтому использую depends_on.",
    
    "CQRS разделяет команды (запись) и запросы (чтение) в разные модели. В Django создаю отдельные view-классы для команд и запросов. Команды идут через один микросервис, а запросы через другой. Использую Django Channels для асинхронной обработки команд и Redis для передачи сообщений между сервисами. Главное синхронизировать данные между read и write моделями."
]


if __name__ == "__main__":
    print("=== АНАЛИЗ СОБЕСЕДОВАНИЯ ===")
    
    # Анализ - ИСПРАВЛЕНО!
    qa_formatted = format_qa_for_analysis(QUESTIONS, ANSWERS)
    analysis = analyze_interview(qa_formatted)  # Вызов функции, а не создание кортежа
    
    print(f"📊 Оценка коммуникации: {analysis['communicationScore']}/10")
    print(f"📝 Общий отзыв: {analysis['overallFeedback']}")
    
    print(f"\n✅ Сильные стороны:")
    for strength in analysis['strengths']:
        print(f"  • {strength}")
        
    print(f"\n⚠️  Области для улучшения:")
    for area in analysis['improvementAreas']:
        print(f"  • {area}")
        
    print(f"\n💬 Подтверждающие цитаты:")
    for quote in analysis['supportingQuotes']:
        print(f"  🔸 \"{quote['quote']}\"")
        print(f"     {quote['analysis']} ({quote['type']})")

=== АНАЛИЗ СОБЕСЕДОВАНИЯ ===
Ошибка декодирования JSON. Ответ от модели: <think>
Хорошо, мне нужно проанализировать коммуникативные навыки кандидата на основе предоставленного транскрипта интервью. Начну с того, что внимательно прочитаю весь транскрипт и выделю ключевые моменты.

Первый вопрос о кэшировании в Django. Кандидат четко объяснил, как настроить Redis, использовать cache.set и cache.get, а также декораторы. Он также упомянул инвалидацию кэша. Это показывает хорошее понимание темы и способность структурировать ответ.

Второй вопрос о HTTP-методах. Кандидат различает PUT и PATCH, объяснил их использование, добавил важность проверки прав доступа и CSRF-токенов. Это демонстрирует не только знания, но и понимание безопасности, что важно для коммуникации.

Третий вопрос о PostgreSQL. Кандидат упомянул EXPLAIN ANALYZE, индексы, типы JOIN и разбивку запросов. Он структурировал ответ, показывая системный подход к оптимизации, что подчеркивает ясность и логичность.

Четвертый вопрос о 

TypeError: 'NoneType' object is not subscriptable